In [89]:
import urllib.request
from bs4 import BeautifulSoup 
import requests
import json 
import re
import pandas as pd

##info
##http://stackoverflow.com/questions/11322430/python-how-to-send-post-request

#adress => en-tete de la requete > referer > @
url = "http://base-donnees-publique.medicaments.gouv.fr/index.php"  

#get the parameters after input>> reseau > parametre > copy the dictionnary
data_post ={'page':"1", 'affliste' :"0", 'affNumero' :"0",'affNumero' :"0",'isAlphabet' :"0",'inClauseSubst' :"0",'nomSubstances' :"",'typeRecherche' :"0",'choixRecherche' :"medicament",'paginationUsed' :"0",'txtCaracteres' :"ibuprofene",'btnMedic.x' :"0",'btnMedic.y' :"0",'radLibelle' :"2",'txtCaracteresSub' :"",'radLibelleSub' :"4"}

r = requests.post(url, data_post )
print(r)

#make soup soup = BeautifulSoup(r,"html.parser")
#res=soup.find_all("td", class_="alignleft") test=  soup.find_all('caption')

#<td class="alignleft" 
  
#nb_medicament =
#molecule,nom du medoc, unite de dosage, forme secable, laboratoire.. dans un dataframe

<Response [200]>


In [6]:
print(r.status_code, r.reason)

200 OK


In [48]:
r.text #donne la nouvelle page avec la nouvelle url mais en moche
type(r.text ) #str
res=BeautifulSoup(r.text, "lxml") #tansform into beautiful
res_medoc =res.find_all("a",class_="standart") #les nom des medoc mais il reste  page 2 et page 3
res_medoc[0]#get one line of the drugs
str(res_medoc[0]) #cast to str
len(res_medoc)# size  
res_medoc

[<a class="standart" href="extrait.php?specid=61349237" title="Cliquez pour accéder à la fiche info et à la liste des documents de référence du médicament :  IBUPROFENE ACTAVIS 400 mg, comprimé pelliculé">IBUPROFENE ACTAVIS 400 mg, comprimé pelliculé				</a>,
 <a class="standart" href="extrait.php?specid=61478851" title="Cliquez pour accéder à la fiche info et à la liste des documents de référence du médicament :  IBUPROFENE ALMUS 200 mg, comprimé pelliculé">IBUPROFENE ALMUS 200 mg, comprimé pelliculé				</a>,
 <a class="standart" href="extrait.php?specid=62611169" title="Cliquez pour accéder à la fiche info et à la liste des documents de référence du médicament :  IBUPROFENE ALMUS 400 mg, comprimé pelliculé">IBUPROFENE ALMUS 400 mg, comprimé pelliculé				</a>,
 <a class="standart" href="extrait.php?specid=61249824" title="Cliquez pour accéder à la fiche info et à la liste des documents de référence du médicament :  IBUPROFENE ARROW 200 mg, comprimé enrobé">IBUPROFENE ARROW 200 mg, com

In [41]:
medoc=re.search('IBUPROFENE ([A-Z ]+) (\d+) ?([\w%]+),([ \w]+)', str(res_medoc[1]))

In [85]:
for i in range(20):
    print(re.search('IBUPROFENE ([A-Z ]+) (\d+) ?([\w%]+),([ \w]+)', str(res_medoc[i])).group(0)) 

IBUPROFENE ACTAVIS 400 mg, comprimé pelliculé
IBUPROFENE ALMUS 200 mg, comprimé pelliculé
IBUPROFENE ALMUS 400 mg, comprimé pelliculé
IBUPROFENE ARROW 200 mg, comprimé enrobé
IBUPROFENE ARROW 400 mg, comprimé pelliculé
IBUPROFENE ARROW 5 %, gel
IBUPROFENE ARROW CONSEIL 400 mg, comprimé pelliculé
IBUPROFENE BIOGARAN 200 mg, comprimé pelliculé
IBUPROFENE BIOGARAN 400 mg, comprimé pelliculé
IBUPROFENE BIOGARAN CONSEIL 200 mg, comprimé pelliculé
IBUPROFENE BIOGARAN CONSEIL 400 mg, comprimé pelliculé
IBUPROFENE CHEMINEAU 5%, solution pour application cutanée
IBUPROFENE CHEMINEAU 5%, solution pour application cutanée en flacon pressurisé
IBUPROFENE CRISTERS 200 mg, comprimé enrobé
IBUPROFENE CRISTERS 200 mg, comprimé pelliculé
IBUPROFENE CRISTERS 400 mg, comprimé pelliculé
IBUPROFENE DU SAINT BERNARD 5 %, gel pour application locale
IBUPROFENE EG 200 mg, comprimé pelliculé
IBUPROFENE EG 400 mg, comprimé pelliculé
IBUPROFENE EVOLUGEN 400 mg, comprimé pelliculé


In [115]:
L=[]
# d = {'col1': ts1, 'col2': ts2}
for i in range(20):
    tmp=re.search('IBUPROFENE ([A-Z ]+) (\d+) ?([\w%]+),([ \w]+)', str(res_medoc[i])).group(0)
    len = tmp.find(" ")
    mol=tmp[:len]
    nom =re.search( ' ([A-Z ]+)',tmp).group(0)
    qte=re.search( '(\d+) ?([\w%]+)',tmp).group(0)
    type=re.search( ',([ \w]+)',tmp).group(0)[1:]
    l = {'nom molecule': mol, 'nom': nom,"quantité":qte,"type":type}
    L.append(l)

In [116]:
df=pd.DataFrame(L)
df

,nom,nom molecule,quantité,type
0,ACTAVIS,IBUPROFENE,400 mg,comprimé pelliculé
1,ALMUS,IBUPROFENE,200 mg,comprimé pelliculé
2,ALMUS,IBUPROFENE,400 mg,comprimé pelliculé
3,ARROW,IBUPROFENE,200 mg,comprimé enrobé
4,ARROW,IBUPROFENE,400 mg,comprimé pelliculé
5,ARROW,IBUPROFENE,5 %,gel
6,ARROW CONSEIL,IBUPROFENE,400 mg,comprimé pelliculé
7,BIOGARAN,IBUPROFENE,200 mg,comprimé pelliculé
8,BIOGARAN,IBUPROFENE,400 mg,comprimé pelliculé
9,BIOGARAN CONSEIL,IBUPROFENE,200 mg,comprimé pelliculé


In [59]:
#re.search( '(\d+)',tmp.group(0))
type(tmp.group(0))

str

In [114]:
re.search( '(\d+) ?([\w%]+)','IBUPROFENE EG 200 mg, comprimé pelliculé')

<_sre.SRE_Match object; span=(14, 20), match='200 mg'>

In [102]:
re.search( ' ([A-Z ]+)','IBUPROFENE CHEMINEAU 5%, solution pour application cutanée')

<_sre.SRE_Match object; span=(10, 21), match=' CHEMINEAU '>

In [103]:
re.search( ',([ \w]+)','IBUPROFENE CHEMINEAU 5%, solution pour application cutanée')

<_sre.SRE_Match object; span=(23, 58), match=', solution pour application cutanée'>

In [65]:
tmp.group(0).find(" ")

10

In [66]:
tmp.group(0)[:10]

'IBUPROFENE'